In [1]:
import pandas as pd
import os

# Loading Data

In [2]:
join_paths = lambda csv_path: os.path.join("datasets", csv_path)

In [3]:
train_path = join_paths("train.csv")
test_path = join_paths("test.csv")

In [4]:
train_path, test_path

('datasets/train.csv', 'datasets/test.csv')

In [5]:
train_df, test_df = pd.read_csv(train_path), pd.read_csv(test_path)

# Train test target split

In [6]:
TARGET = "banking_crisis"

In [7]:
def xy_split(df, target_column):
    # Return x and then y
    x = df.drop(columns=target_column)
    y = df[target_column]
    return x, y

In [8]:
X_train, y_train = xy_split(train_df, TARGET)
X_test, y_test = xy_split(test_df, TARGET)

In [9]:
print(X_train.columns)
print("*"*79)
print(y_train.name)

Index(['Latitude (average)', 'Longitude (average)', 'currency_crises',
       'domestic_debt_in_default', 'exch_usd', 'gdp_weighted_default',
       'independence', 'inflation_annual_cpi', 'inflation_crises',
       'sovereign_external_debt_default', 'systemic_crisis', 'year'],
      dtype='object')
*******************************************************************************
banking_crisis


-----

In [10]:
X_train.describe()

,Latitude (average),Longitude (average),currency_crises,domestic_debt_in_default,exch_usd,gdp_weighted_default,independence,inflation_annual_cpi,inflation_crises,sovereign_external_debt_default,systemic_crisis,year
count,952.000000,952.000000,952.000000,952.000000,952.000000,952.000000,952.000000,952.000000,952.000000,952.000000,952.000000,952.000000
mean,0.525743,0.412352,0.133403,0.037815,0.385582,0.006607,0.769958,0.465231,0.132353,0.150210,0.075630,0.700658
std,0.355233,0.260056,0.352336,0.190849,0.344868,0.044272,0.421081,0.051934,0.339052,0.357465,0.264545,0.214604
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.142857,0.207834,0.000000,0.000000,0.071994,0.000000,1.000000,0.458406,0.000000,0.000000,0.000000,0.590909
50%,0.571429,0.463629,0.000000,0.000000,0.285457,0.000000,1.000000,0.465127,0.000000,0.000000,0.000000,0.733766
75%,0.888889,0.559552,0.000000,0.000000,0.671914,0.000000,1.000000,0.473414,0.000000,0.000000,0.000000,0.870130
max,1.000000,1.000000,2.000000,1.000000,1.000000,0.400000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
